# Trend Following Crypto Trading Bot

In [97]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from datetime import datetime, timedelta
import ccxt
from config import *
import time

In [94]:
def get_price(symbol, api):
    return api.fetch_ticker(symbol)['last']
    
def get_data(symbol, timeframe, api, number_of_days=30):
    """ TimeFrame examples: "1m", "15m", "1h"..."""
    
    # Initialiaze the Exchange
    api.load_markets()
    
    # Define the dates & import the data
    one_week_ago = datetime.utcnow() - timedelta(days=number_of_days)
    since = int(one_week_ago.timestamp() * 1000)
    data = api.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=1000)

    # Format the data
    df_data = pd.DataFrame(data)
    df_data.columns = ["time", "open", "high", "low", "close", "volume"]
    df_data["time"] = pd.to_datetime(df_data["time"], unit='ms')
    df_data = df_data.set_index("time")
    
    return df_data

def buy_market(symbol, symbol_amount, api):
    price = get_price(symbol, api)
    symbol_amount_usdt = symbol_amount * price
  
    # You maybe need to modify the inputs of this function according to your exchange
    order = api.create_market_buy_order(symbol, symbol_amount_usdt)
    return order

def sell_market(symbol, symbol_amount, api):
    order = api.create_market_sell_order(symbol, symbol_amount)
    return order

def get_available_amount(symbol,api):
    balances = api.fetch_balance()
    for dictionnary in balances["info"]:
        if dictionnary["coin"]==symbol:
            amount_symbol = dictionnary["available"]
        else:
            amount_symbol = 0
    return float(amount_symbol)

In [108]:
balances = api.fetch_balance()
balances["info"]

[{'coin': 'BTC',
  'available': '0.00000081',
  'limitAvailable': '0',
  'frozen': '0.00000000',
  'locked': '0.00000000',
  'uTime': '1728293413716'},
 {'coin': 'USDT',
  'available': '55.18492785',
  'limitAvailable': '0',
  'frozen': '0.00000000',
  'locked': '0.00000000',
  'uTime': '1734514475558'},
 {'coin': 'DOGE',
  'available': '0.00006260',
  'limitAvailable': '0',
  'frozen': '0.00000000',
  'locked': '0.00000000',
  'uTime': '1734514475558'}]

<br>

### Test the Functions

In [106]:
get_price("BTCUSDT", api)

104987.31

In [107]:
get_data("BTCUSDT", "1h", api)

,open,high,low,close,volume
time,,,,,
2024-11-18 11:00:00,91737.83,91792.68,90062.50,90264.54,138.270318
2024-11-18 12:00:00,90264.54,90740.74,90111.13,90517.51,90.896266
2024-11-18 13:00:00,90517.51,90583.28,89400.00,89526.09,119.259774
2024-11-18 14:00:00,89526.09,91031.28,89452.06,90305.95,149.818628
2024-11-18 15:00:00,90305.95,92509.63,90247.64,92272.70,269.969662
...,...,...,...,...,...
2024-12-18 07:00:00,103815.99,104340.00,103756.00,104300.26,438.659404
2024-12-18 08:00:00,104300.26,104402.16,103789.00,103848.01,409.670293
2024-12-18 09:00:00,103848.01,104426.45,103841.01,104091.00,235.189755


In [117]:
buy_market("DOGEUSDT", 20, api)
balances = api.fetch_balance()
balances["info"]

[{'coin': 'BTC',
  'available': '0.00000081',
  'limitAvailable': '0',
  'frozen': '0.00000000',
  'locked': '0.00000000',
  'uTime': '1728293413716'},
 {'coin': 'USDT',
  'available': '47.36137541',
  'limitAvailable': '0',
  'frozen': '0.00000000',
  'locked': '0.00000000',
  'uTime': '1734523958979'},
 {'coin': 'DOGE',
  'available': '19.98007300',
  'limitAvailable': '0',
  'frozen': '0.00000000',
  'locked': '0.00000000',
  'uTime': '1734523958979'}]

In [111]:
sell_market("DOGEUSDT", 19.97386880, api)
balances = api.fetch_balance()
balances["info"]

[{'coin': 'BTC',
  'available': '0.00000081',
  'limitAvailable': '0',
  'frozen': '0.00000000',
  'locked': '0.00000000',
  'uTime': '1728293413716'},
 {'coin': 'USDT',
  'available': '55.17174372',
  'limitAvailable': '0',
  'frozen': '0.00000000',
  'locked': '0.00000000',
  'uTime': '1734523476430'},
 {'coin': 'DOGE',
  'available': '0.00006880',
  'limitAvailable': '0',
  'frozen': '0.00000000',
  'locked': '0.00000000',
  'uTime': '1734523476430'}]

In [114]:
get_available_amount("DOGE",api)

19.975873

### Create the bot

In [118]:
amount_symbol = 20

while True:
    # Import and Create the necessary data 
    df = get_data("DOGEUSDT", "1h", api).iloc[:-1,:]
    df["Fast SMA"] = df["close"].rolling(24*2).mean()
    df["Slow SMA"] = df["close"].rolling(24*20).mean()

    # Create the conditions
    fast_sma, slow_sma = df["Fast SMA"].iloc[-1], df["Slow SMA"].iloc[-1]
    print(fast_sma, slow_sma)
    condition_buy = slow_sma < fast_sma
    condition_sell = not condition_buy

    # Extract the data
    av_amount = get_available_amount("DOGE", api)

    # Buy condition
    if condition_buy and (av_amount < amount_symbol*.10):
        buy_market("DOGEUSDT", amount_symbol, api)
        print("BUY")

    # Sell condition
    elif condition_sell and (amount_symbol*.90 < av_amount):
        sell_market("DOGEUSDT", av_amount, api)
        print("SELL")

    else:
        pass

    time.sleep(1)

0.3980818750000001 0.4181456041666667
SELL
0.3980818750000001 0.4181456041666667
0.3980818750000001 0.4181456041666667


KeyboardInterrupt: 